In [1]:
import os
import boto3
import requests
import pandas as pd
import pyarrow as pa
from io import BytesIO
import pyarrow.parquet as pq
from datetime import datetime

In [2]:
API_LINK = f"https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad"

In [3]:
requisicao = requests.get(API_LINK)

In [4]:
wheater_json = requisicao.json()

In [5]:
weather_df = pd.json_normalize(wheater_json)

In [6]:
weather_df

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 804, 'main': 'Clouds', 'description': ...",stations,10000,1701656772,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,...,1011,1.45,107,6,100,2,2020383,BR,1701591403,1701641526


In [7]:
df_lon_lat = pd.DataFrame(wheater_json['coord'], index=[0])

In [8]:
df_lon_lat

,lon,lat
0,-51.1306,-29.6783


In [9]:
df_weather = pd.DataFrame(wheater_json['weather'], index=[0])

In [10]:
df_weather

,id,main,description,icon
0,804,Clouds,overcast clouds,04n


In [11]:
df_main = pd.DataFrame(wheater_json['main'], index=[0])

In [12]:
df_main

,temp,feels_like,temp_min,temp_max,pressure,humidity,sea_level,grnd_level
0,294.34,294.89,293.7,295.62,1015,91,1015,1011


In [13]:
selected_columns = [
    "base",
    "visibility",
    "dt",
    "timezone",
    "id",
    "name",
    "cod",
    "coord.lon",
    "coord.lat",
    "main.temp",
    "main.feels_like",
    "main.temp_min",
    "main.temp_max",
    "main.pressure",
    "main.humidity",
    "main.sea_level",
    "main.grnd_level",
    "wind.speed",
    "wind.deg",
    "wind.gust",
    "clouds.all",
    "sys.type",
    "sys.id",
    "sys.country",
    "sys.sunrise",
    "sys.sunset",
]

In [14]:
weather_df = pd.DataFrame(weather_df)

In [15]:
weather_df

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 804, 'main': 'Clouds', 'description': ...",stations,10000,1701656772,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,...,1011,1.45,107,6,100,2,2020383,BR,1701591403,1701641526


In [16]:
df_clumns_selected = weather_df[selected_columns]

In [17]:
df_clumns_selected

,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,stations,10000,1701656772,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,294.34,...,1011,1.45,107,6,100,2,2020383,BR,1701591403,1701641526


In [18]:
super_df = pd.concat([df_weather, df_lon_lat, df_main, df_clumns_selected], axis=0, ignore_index=True)

In [19]:
super_df

,id,main,description,icon,lon,lat,temp,feels_like,temp_min,temp_max,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,804.0,Clouds,overcast clouds,04n,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,-51.1306,-29.6783,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,294.34,294.89,293.7,295.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3456068.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1011.0,1.45,107.0,6.0,100.0,2.0,2020383.0,BR,1.701591e+09,1.701642e+09


In [20]:
super_df.to_parquet("C:/Users/crist/Desktop/s3_testes/super_df.parquet")

Save at S3

In [21]:
# Defina suas credenciais explicitamente (não recomendado)
profile_name = 'CRISTIAN_AWS'

s3 = boto3.session.Session(profile_name=profile_name).client('s3')

In [22]:
bucket_name = 'cristian-teste-buckt-from-python-code'

In [23]:
local_file_path = "C:/Users/crist/Desktop/s3_testes/super_df.parquet"

In [24]:
dia_da_exucucao = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

In [25]:
dia_da_exucucao

'2023-12-03-23-27-40'

In [26]:
s3_file_name = f'df_teste_{dia_da_exucucao}.parquet'

In [27]:
s3_file_name

'df_teste_2023-12-03-23-27-40.parquet'

In [28]:
s3.upload_file(local_file_path, bucket_name, s3_file_name)

---------------------------------

CATUABA